In [ ]:
import re

import numpy as np
import pandas as pd
from multielo import Tracker

In [ ]:
fixtures = pd.read_parquet("../data/cleansed/fixtures.parquet")
plays = pd.read_parquet("../data/cleansed/plays.parquet")

# features stores
fts_shift, fts_nshift = {}, []

#### Fixtures-wise

In [ ]:
# normalise odds to ratio
mkts = ["rate_mkt_h", "rate_mkt_d", "rate_mkt_a"]
odds = ["avgch", "avgcd", "avgca"]

fixtures[mkts] = 1 / fixtures[odds]
fixtures[mkts] = fixtures[mkts].div(fixtures[mkts].sum(axis=1), axis=0)

fts_nshift.extend(["rate_mkt_h", "rate_mkt_a"])

In [ ]:
# apply insider knowledge for market intelligence
fixtures["knwl_mkt_intel"] = np.select(
    [fixtures["rate_mkt_h"] >= .4, fixtures["rate_mkt_a"] >= .3],
    ["H", "A"],
    default="D",
)

fts_nshift.append("knwl_mkt_intel")

In [ ]:
# categorise time frame
fixtures["timestamp"] = pd.to_datetime(fixtures["time"], format="%H:%M")

fixtures["time_frame"] = np.where(
    fixtures["timestamp"].dt.hour < 18,
    "noon", "night",
)

fts_nshift.append("time_frame")

In [ ]:
# formulate head to head (stadium dependent) win and lose rate
fixtures["teams"] = fixtures[["home", "away"]].apply(tuple, axis=1)
fixtures["n_h2h_game"] = fixtures.groupby("teams")["date"].cumcount() + 1

for res, val in {"win": "H", "lose": "A"}.items():
    fixtures[f"n_{res}"] = fixtures.groupby("teams", as_index=False)["res"] \
        .apply(lambda x: (x==val).cumsum()) \
        .reset_index() \
        .sort_values(by="level_1", ignore_index=True)["res"]
    
    fixtures[f"rate_h2h_{res}"] = fixtures[f"n_{res}"] / fixtures["n_h2h_game"]

fts_shift.update({"rate_h2h_win": .4, "rate_h2h_lose": .3})

In [ ]:
# apply insider knowledge for head to head rate
fixtures["rate_h2h_diff"] = fixtures["rate_h2h_win"] - fixtures["rate_h2h_lose"]

fixtures["knwl_h2h"] = np.select(
    [fixtures["rate_h2h_diff"] > .3, fixtures["rate_h2h_diff"] < -.2],
    ["H", "A"],
    default="D",
)

fts_shift.update({"knwl_h2h": "H"})

In [ ]:
col = [
    "season", "date", "time", "time_frame",
    "home", "away", "res",
    "avgch", "avgcd", "avgca", "knwl_mkt_intel",
    "rate_mkt_h", "rate_mkt_d", "rate_mkt_a",
    "teams", "rate_h2h_win", "rate_h2h_lose", "knwl_h2h",
]
fixtures = fixtures[col].copy()

In [ ]:
fixtures.to_parquet("../data/featured/features.parquet")

#### Plays-wise

In [ ]:
# formulate seasonal rest days
plays["dt"] = pd.to_datetime(plays["date"])
plays["dt_next"] = plays.groupby(["season", "team"])["dt"].shift(-1)

plays["n_rest_day"] = plays["dt_next"] - plays["dt"]
plays["n_rest_day"] = plays["n_rest_day"].dt.days
plays["n_rest_day"] = plays["n_rest_day"].where(plays["n_rest_day"] < 14, 7) \
    .astype(int)

fts_nshift.append("n_rest_day")

In [ ]:
# formulate seasonal win and lose rate
plays["n_game"] = plays.groupby(["season", "team"])["date"].cumcount() + 1

for res, val in {"win": 3, "lose": 0}.items():
    plays[f"n_{res}"] = plays.groupby(["season", "team"])["points"] \
        .apply(lambda x: (x==val).cumsum()) \
        .reset_index() \
        .sort_values(by="level_2", ignore_index=True)["points"]
    
    plays[f"rate_seas_{res}"] = plays[f"n_{res}"] / plays["n_game"]

fts_shift.update({"rate_seas_win": .4, "rate_seas_lose": .3})

In [ ]:
# accumulate seasonal scores
plays["scores"] = plays.groupby(["season", "team"])["points"].cumsum()

fts_shift.update({"scores": 0})

In [ ]:
# formulate seasonal rank
# create a rank template for all teams
season_date = plays[["season", "date"]].drop_duplicates().values
teams = plays[["team"]].drop_duplicates()

tpls = []
for season, date in season_date:
    tpl = teams.copy()
    tpl[["season", "date"]] = season, date
    tpls.append(tpl)

ranks = pd.concat(tpls, ignore_index=True)
ranks = ranks[["season", "date", "team"]]

# accumulate goals and net goals
goals = ["goals", "net_goals"]

scores = plays[["season", "date", "team", "scores"]].copy()
scores[goals] = plays.groupby(["season", "team"]) \
    [goals].cumsum()

ranks = ranks.merge(scores, how="left")

# rank sort by score, goals and net_goals
criteria = ["scores", "net_goals", "goals"]

ranks[criteria] = ranks.groupby(["season", "team"])[criteria].ffill() \
    .fillna(float("-inf"))

ranks["criteria"] = ranks[criteria].apply(tuple, axis=1)

ranks["rank"] = ranks.groupby(["season", "date"])["criteria"] \
    .rank(method="min", ascending=False) \
    .astype(int)

plays = plays.merge(ranks[["date", "team", "rank"]])

fts_shift.update({"rank": 0})

In [ ]:
# formulate seasonal rating
results = fixtures[["season", "date"]].copy()

results["winner"] = np.select(
    [fixtures["res"] == "H", fixtures["res"] == "A"],
    [fixtures["home"], fixtures["away"]],
    default=fixtures[["home", "away"]].apply(tuple, axis=1),
)

results["loser"] = np.select(
    [fixtures["res"] == "A", fixtures["res"] == "H"],
    [fixtures["home"], fixtures["away"]],
    default=None,
)

dfs = []
for season in results["season"].unique():
    tracker = Tracker()
    tracker.process_data(
        results.query(f"season == {season}")[["date", "winner", "loser"]]
    )
    
    elo_season = tracker.get_history_df()
    elo_season.rename(
        columns={"player_id": "team", "rating": "rating_seas"},
        inplace=True,
    )
    
    dfs.append(elo_season)

elo = pd.concat(dfs, ignore_index=True)

plays = plays.merge(elo)

fts_shift.update({"rating_seas": 1_000})

In [ ]:
# formulate historical rank
results = fixtures[["season", "date"]].copy()

results["winner"] = np.select(
    [fixtures["res"] == "H", fixtures["res"] == "A"],
    [fixtures["home"], fixtures["away"]],
    default=fixtures[["home", "away"]].apply(tuple, axis=1),
)

results["loser"] = np.select(
    [fixtures["res"] == "A", fixtures["res"] == "H"],
    [fixtures["home"], fixtures["away"]],
    default=None,
)

tracker = Tracker()
tracker.process_data(results[["date", "winner", "loser"]])

elo = tracker.get_history_df()
elo.rename(
    columns={"player_id": "team", "rating": "rating_hist"},
    inplace=True,
)

plays = plays.merge(elo)

fts_shift.update({"rating_hist": 1_000})

In [ ]:
col = [
    "season", "date", "team",
    "goals", "net_goals", "points",
    "n_game", "rank", "scores",
    "rate_seas_win", "rate_seas_lose",
    "rating_seas", "rating_hist",
    "stadium", "n_rest_day",
]
plays = plays[col].copy()

In [ ]:
plays.to_parquet("../data/featured/plays.parquet")

#### Feature Shift

In [ ]:
col = [col for col in fixtures.columns if col in fts_shift.keys()]

fixtures[col] = fixtures.groupby("teams")[col].shift()
fixtures.fillna(fts_shift, inplace=True)

In [ ]:
col = [col for col in plays.columns if col in fts_shift.keys()]

plays[col] = plays.groupby(["season", "team"])[col].shift()
plays.fillna(fts_shift, inplace=True)

#### Denormalisation

In [ ]:
fts = fts_nshift + list(fts_shift.keys())

col = [
    "season", "date", "res",
    "home", "away",
] + [
    col for col in fixtures.columns
    if col in fts
]
games = fixtures[col].copy()

col = [
    "date", "team",
] + [
    col for col in plays.columns
    if col in fts
]
teams = plays[col].copy()

In [ ]:
j1 = games.merge(
    teams.rename(columns={"team": "home"}),
    how="left",
    on=["date", "home"],
).merge(
    teams.rename(columns={"team": "away"}),
    how="left",
    on=["date", "away"],
    suffixes=("_h", "_a"),
)

In [ ]:
col_h = [col for col in j1.columns if col.endswith("_h")]
col_a = [col for col in j1.columns if col.endswith("_a")]

col = [re.sub("_h$", "_net", col) for col in col_h]
j1[col] = j1[col_h].values - j1[col_a].values

col = ["date", "home", "away"] + col_h + col_a
j1.drop(columns=col, inplace=True)

In [ ]:
# convert HDA to handicap result
# `rank_net > 0` means home team has better season ranking
# in general handicap favorites to home team
# if the game drew meaning away team won the handicap, vice versa
hcap = np.where(j1["rank_net"] > 0, "A", "H")
j1["res"] = j1["res"].where(j1["res"] != "D", hcap)

In [ ]:
j1.to_parquet("../data/featured/j1_league.parquet")